# Installations

In [1]:
# Check how to install pybydump
# !pip install neo4j

# Importing packages

In [2]:
from neo4j import GraphDatabase
from pathlib import Path

# Load Google Drive

In [3]:
#Allowing Google Drive access
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
path = '/content/drive/MyDrive/2023_capstone/06. Codes/00. data'
dataPath = Path(path)

# Getting the data

In [5]:
# Define the path to your graph data file
data_file_path = dataPath / "oregon2_010331.txt"

In [6]:
f = open(data_file_path, 'r')
lines = f.readlines()[4:]
lines = [line.strip('\n').split('\t') for line in lines]

##### We will need to modify this section once we know how to access BGP data

# Creating the Graph Database

In [7]:
# Define a function to create the graph in Neo4j
def create_graph(tx, from_node_id, to_node_id):
    query = (
        "MERGE (fromNode:Node {id: $from_node_id}) "
        "MERGE (toNode:Node {id: $to_node_id}) "
        "MERGE (fromNode)-[:CONNECTS]->(toNode)"
    )
    tx.run(query, from_node_id=from_node_id, to_node_id=to_node_id)

In [8]:
# Connect to the Neo4j database
uri = "bolt://34.200.229.137:7687"  # Change to match your Neo4j server settings
username = "neo4j"      # Change to your Neo4j username
password = "margin-branches-shock"      # Change to your Neo4j password

# Create a Neo4j driver
driver = GraphDatabase.driver(uri, auth=(username, password))

In [ ]:
with driver.session() as session:
  for line in lines:
      from_node_id, to_node_id = map(int, line)
      session.write_transaction(create_graph, from_node_id, to_node_id)

<ipython-input-9-ceb066031dd3>:4: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_graph, from_node_id, to_node_id)


In [ ]:
# Close the Neo4j driver
driver.close()